In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

In [2]:
import sys
import os

In [3]:
def flushprint(www):
    sys.stdout.write('\r')
    sys.stdout.write('%s' %www)
    sys.stdout.flush()

### 城市餐厅类型及数量

In [4]:
food_url='https://www.tripadvisor.cn/Restaurants-g294212-Beijing.html'

In [5]:
html=requests.get(food_url)
html.encoding='utf-8'
content=html.text
soup=BeautifulSoup(content,'html.parser')

In [6]:
page_url=[]
for page in soup.select('.pageNumbers a'):
    page_url.append(page['href'])

In [8]:
food=[]

In [9]:
# 最后一页的offset=12330，一页30条数据，模拟翻页爬取数据
for page in range(0,12330,30):
    try:        
        if page==0: 
            url=food_url
            tt = random.random()
            flushprint(str(page)+':'+str(url)+':sleeping'+str(tt)+'seconds')
            time.sleep(tt)
            for item in soup.select('.restaurants-list-ListCell__infoWrapper--3agHz .restaurants-list-ListCell__nameBlock--1hL7F a'):
                food_name=item.text
                food_link=item['href']
                food_=[food_name,food_link]
                food.append(food_)
        else:
            url='https://www.tripadvisor.cn/Restaurants-g294212-oa'+str(page)+'-Beijing.html#EATERY_LIST_CONTENTS'            
            tt = random.random()
            flushprint(str(page)+':'+str(url)+':sleeping'+str(tt)+'seconds')
            time.sleep(tt)
            
            html=requests.get(url)
            html.encoding='utf-8'
            content=html.text
            soup=BeautifulSoup(content,'html.parser')
                        
            for item in soup.select('.restaurants-list-ListCell__infoWrapper--3agHz .restaurants-list-ListCell__nameBlock--1hL7F a'):
                food_name=item.text
                food_link=item['href']
                food_=[food_name,food_link]
                food.append(food_)
    except Exception as e:
        print(page,url,e)

12300:https://www.tripadvisor.cn/Restaurants-g294212-oa12300-Beijing.html#EATERY_LIST_CONTENTS:sleeping0.42205268271460694secondss

In [11]:
df1=pd.DataFrame(food,columns=['food-name','food-link'])

In [12]:
df1.to_excel('beijing_food.xls')

### 城市具体餐厅评分数据

In [13]:
food=pd.read_excel('beijing_food.xls')

In [14]:
food_score=[]

In [ ]:
for index in range(0,12303):
    food_name=food.iloc[index]['food-name']
    food_link=food.iloc[index]['food-link']
    url='https://www.tripadvisor.cn'+food.iloc[index]['food-link']
    try:
        tt = random.random()
        flushprint(str(index)+':'+str(url)+':sleeping'+str(tt)+'seconds')
        time.sleep(tt)
        
        html=requests.get(url)
        html.encoding='utf-8'
        content=html.text
        soup=BeautifulSoup(content,'html.parser')
        
        if soup.select('#taplc_resp_rr_top_info_rr_resp_0 .ratingContainer .ui_bubble_rating'):
            score=soup.find('span',class_='ui_bubble_rating')['class'][1].replace('bubble_','')
        else:
            score=0
            
        if soup.select('#taplc_resp_rr_top_info_rr_resp_0 .ratingContainer .reviewCount'):
            review_num=soup.find('span',class_='reviewCount').text
        else:
            review_num=0
        food_score_=[food_name,food_link,score,review_num]
        food_score.append(food_score_)
        
    except Exception as e:
        print(index,url,e)
        continue

In [25]:
df2=pd.DataFrame(food_score,columns=['food-name','food-link','food-score','review-num'])

In [26]:
df2.to_excel('beijing_score.xls')